Goal:

1. Create new AWS instance with Postgres and anaconda. 

2. Create function which take urlname, date (with default set to Jan 1, 2018, and inserts these things to Mongo:
    1. GroupID
    2. Status
    3. urlname
    4. members
    5. created
    6. city
    7. join_mode- This can be approval, closed, or open. Are closed groups, do they have higher retentions? Or approval?
    8. category.id
    9. meta_category.id
    10. Description?
    11. State
    12. lat
    13. lon
    14. group_photo.photo_link
    15. join_info.questions_req : Says whether any questions are required when joining group. This could cause: Less people to show up, but more to come back 
    16. join_info.photo_req
    17. join_info.questions
    18. contributions: can people 'contribute' money? 
    19. membership_dues.fee
    20. membership_dues.fee_desc (How often fee due)
    21. membership_dues.required
    22. past_event_count
    23. topics
    24. who (what they call members)
    
Let's make two databases, one on group, another on events. 

Or the function could take an area and a date. 

## Austin:

I say central zipcode is 78702. 12 miles seems to encompass the city nicely. Let's do 17 to be safe. Well, when you think about it, don't want to get meetups too far away.

In [6]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import requests
import json
import time
import sys

sys.path.append('./src')
key = open('src/api.txt', 'r').read()

client = MongoClient()
db = client.meetup

In [39]:
def get_attendees_for_group_and_time(zipcode, radius):
    '''
    Finds all groups within a certain region. If their visibility is set to public, 
    insert them into the mongo groups collection in the meetup db. 
    
    Parameters:
    zipcode (int): Center of search for groups.
    radius(int): Distance past zip to search for groups.

    '''
    req = requests.get('https://api.meetup.com/find/groups?photo-host=private&key={}&zip={}&radius={}\
    &fields=join_info,contributions,discussion_preferences,membership_dues,\
    best_topics,past_event_count,topics,plain_text_description&page=200&offset={}\
    &omit=link,score,name,description,country,organizer,timezone,next_event,localized_country_name,localized_location,untranslated_city,meta_category.photo'
                       .format(key, zipcode, radius, 0))
    json_data = json.loads(req.text)

    #This will give us the number of groups.
    number_of_groups = int(req.headers['X-Total-Count'])
    offsets = list(range(int(number_of_groups/200) + 1))
    return number_of_groups, offsets   
    
get_attendees_for_group_and_time(78702, 12)

ConnectionError: HTTPSConnectionPool(host='api.meetup.com', port=443): Max retries exceeded with url: /find/groups?photo-host=private&key=3669354f2b4a715c4f25f6191d7f6e&zip=78702&radius=12%20%20%20%20&fields=join_info,contributions,discussion_preferences,membership_dues,%20%20%20%20best_topics,past_event_count,topics,plain_text_description&page=200&offset=0%20%20%20%20&omit=link,score,name,description,country,organizer,timezone,next_event,localized_country_name,localized_location,untranslated_city,meta_category.photo (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5b2008bdd8>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
def get_events_for_group(starting_date):
    pass